In [1]:
from pm4py.objects.log.importer.xes import factory as xes_importer
from pm4py.objects.log.importer.csv import factory as csv_importer

from pm4py.algo.discovery.dfg import factory as dfg_factory
from pm4py.visualization.dfg import factory as dfg_vis_factory

import pandas as pd

import timeit
import os

In [2]:
file = '/data/BPI2016_Clicks_NOT_Logged_In.csv'
experiment1_file = '/data/experiment1.csv'
method = 'pm4py'

In [3]:
CPU_Limit = os.getenv('CPU_LIMIT')
MEMORY_Limit = os.getenv('MEMORY_LIMIT')

In [4]:
result_columns = ['Analysis', 'CPU Limit', 'MEMORY Limit', 'Loading Time', 'DFG Calculation Time', 'Graph Creation Time', 'Total Time']

if os.path.isfile(experiment1_file):
    result = pd.read_csv(experiment1_file)
else:
    result = pd.DataFrame(columns=result_columns)

In [5]:
def getDFG(result, method, CPU_Limit, MEMORY_Limit, file):
    time_start = timeit.default_timer()

    log = pd.read_csv(file, sep=';', encoding='latin-1', usecols=["SessionID", "PAGE_NAME", "TIMESTAMP"])
    log = log.rename(columns={"SessionID": "case:concept:name", "PAGE_NAME": "concept:name", "TIMESTAMP": "time:timestamp"})
    time_log = timeit.default_timer()

    dfg = dfg_factory.apply(log)
    time_dfg = timeit.default_timer()
    
    gviz = dfg_vis_factory.apply(dfg)
    #dfg_vis_factory.view(gviz)
    time_gviz = timeit.default_timer()

    Loading_Time = time_log-time_start
    DFG_Calculation_Time = time_dfg-time_log
    GViz_Calculation_Time = time_gviz-time_dfg
    
    Total_Time = time_gviz-time_start

    new_row = pd.DataFrame([[method, CPU_Limit, MEMORY_Limit, Loading_Time, DFG_Calculation_Time, GViz_Calculation_Time, Total_Time]]
                           , columns=result_columns)

    result = pd.concat([result, new_row])
    
    return result

In [6]:
result = getDFG(result, method, CPU_Limit, MEMORY_Limit, file)

In [7]:
result

,Analysis,CPU Limit,MEMORY Limit,Loading Time,DFG Calculation Time,Graph Creation Time,Total Time
0,pm4py,0.5,2024m,27.09338,25.905822,1.391319,54.390521
